### Equipo de Trabajo:
Kevin Martínez Gallego; Mateo Llano Avendaño; Deiry Sofía Navas

## Punto 4 Taller Spark

### Numeral A
#### Revisión de calidad de los datos

Los datos proporcionados presentan diferentes inconvenientes al momento de ser procesados. Uno de ellos se evidencia en la forma como se separa la información: en algunos casos es por medio de comas(,), en otros casos por medio de (¬) y de (|); esto, aunque no es un problema que impida realizar el proceso de análisis de la información, causa que sea necesario realizar procesos adicionales para llegar al resultado final. Otros problema radica en la variación de las extensiones de los archivos, siendo unos ".txt", otros ".TXT" y otros ".csv".
Tambien, se encontraron inconvenientes en el nombre de las columnas, ya que en algunos casos cambian a través de los años o en otros casos desaparecen.
La mayoria de la información se encuentra documentada y el diccionario proporcionado permite entender el significado de cada variable y el tipo, lo cual agiliza el proceso de entender la información y saber por cuales variables se deben hacer los procesos con SQL.

### Numeral B
Ranking de los colegios por años, de mayor puntaje a menor puntaje, utilizanddo los archivos de pruebas saber.

In [4]:
# Imports ans installations
!pip install -q findspark
import findspark
from IPython.core.debugger import set_trace
from pyspark.sql.functions import desc, expr, asc , regexp_replace, col, lit, row_number 
from pyspark.sql.window import Window
from pyspark.sql import SparkSession , SQLContext
from pyspark import SparkContext,SparkConf
import pandas as pd
findspark.init()

# Configurations
conf = SparkConf().setAppName("ICFES").setMaster("local[*]")
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [5]:
#Base methods

# Split a word by selector indicated
def replace(k,selector):
    word = k.split(selector)
    new_word = [k.split('"')[1] if '"' in k else k for k in word ]
    return new_word


# Replace the "," for "." in the average sobjects
def cast_value(df, columns):
    for column in columns:
        df = df.withColumn(column, regexp_replace(column, ',', '.'))
    return df


# create the sql query to calculate the average total
def toSql(year, columns):
    sqlString = '('
    bln = True
    for x in columns:
        if bln : 
            sqlString +=  x
            bln = False      
        sqlString += ' + ' + x
    sqlString += ") / {0}".format(len(columns)+1)
    return(sqlString)


# Set the correct columns in relation to the year
def classify_columns(year):
    if (year >=20151):
        columns = ['PUNT_LECTURA_CRITICA', 'PUNT_MATEMATICAS', 'PUNT_C_NATURALES', 'PUNT_SOCIALES_CIUDADANAS', 'PUNT_INGLES']
    elif(year>=20071 and year <=20112):
        columns = [ 'punt_lenguaje', 'punt_matematicas', 'punt_c_sociales', 'punt_filosofia', 'punt_biologia', 'punt_quimica', 'punt_fisica', 'punt_ingles']
    elif(year>=20121 and year <=20132):
        columns = ['PUNT_MATEMATICAS', 'PUNT_INGLES', 'PUNT_CIENCIAS_SOCIALES', 'PUNT_BIOLOGIA', 'PUNT_FILOSOFIA', 'PUNT_FISICA', 'PUNT_QUIMICA', 'PUNT_LENGUAJE']
    elif(year>=20141 and year <=20142):  
        columns = ['PUNT_LECTURA_CRITICA', 'PUNT_MATEMATICAS', 'PUNT_C_NATURALES', 'PUNT_SOCIALES_CIUDADANAS', 'PUNT_RAZONA_CUANTITATIVO', 'PUNT_COMP_CIUDADANA', 'PUNT_INGLES']
    elif(year>=20061 and year <=20062):  
        columns = ['punt_lenguaje', 'punt_matematicas', 'punt_c_sociales', 'punt_filosofia', 'punt_biologia', 'punt_quimica', 'punt_fisica', 'punt_idioma']
    return columns


In [6]:
# Automation 2006-1 -2019-1

#Set the query to get the avg info group by cole_nombre_sede
def position_pro_calculation(df_year):
    count = df_year.withColumn("num", lit(1)).groupBy("cole_nombre_sede").sum("num").withColumnRenamed("sum(num)", "Estudiantes").withColumnRenamed("cole_nombre_sede", "col")
    avg = df_year.groupBy("cole_nombre_sede").avg("Prom_Puntaje")
    w = Window().orderBy(desc("avg(Prom_Puntaje)"))
    avg = avg.withColumn("Ranking", row_number().over(w))
    result =count.join(avg, count.col == avg.cole_nombre_sede).select("Ranking","cole_nombre_sede","Estudiantes","avg(Prom_Puntaje)").orderBy(desc("avg(Prom_Puntaje)"))
    result.show()

    
# Load the document in the route
def load_document(init_year):
    try:
        file = f"./SB/SB11_{str(init_year)}.txt"
        saber_pro=sc.textFile(file)
        header = saber_pro.first()
    except Exception as e:
        file = f"./SB/SB11_{str(init_year)}.TXT"
        saber_pro=sc.textFile(file)
        header = saber_pro.first()
    return saber_pro, header


# automation by years 2006-2019
def automation_saber_pro():
    init_year = 20061
    end_year=20191
    while (init_year<=end_year):
        columns = classify_columns(init_year)
        if(init_year%2 == 0):
            saber_pro, header = load_document(init_year)
            print("RESULTADOS PRUEBAS " + str(int(init_year/10)))
            values = saber_pro.map(lambda k: replace(k,'¬'))
            df_2=values.toDF(header.split('¬'))
            
            if(init_year == 20112):
                df_2 = df_2.withColumnRenamed('PUNT_CIENCIAS_SOCIALES', 'punt_c_sociales')
            df_2 = cast_value(df_2,columns)
            df_2=df_2.select('COLE_NOMBRE_SEDE', expr(toSql(init_year, columns) + ' as Prom_Puntaje'))
            df_year = df_1.union(df_2)
            position_pro_calculation(df_year)
            init_year = init_year + 9
        else:
            saber_pro, header = load_document(init_year)
            values = saber_pro.map(lambda k: replace(k,'¬'))
            df_1=values.toDF(header.split('¬'))
            if(init_year == 20141):
                df_1 =df_1.withColumnRenamed('PUNT_CIENCIAS_SOCIALES', 'PUNT_SOCIALES_CIUDADANAS')
                df_1 =df_1.withColumnRenamed('PUNT_BIOLOGIA', 'PUNT_C_NATURALES')
                df_1 =df_1.withColumnRenamed('PUNT_LENGUAJE', 'PUNT_LECTURA_CRITICA')
                df_1 =df_1.withColumnRenamed('PUNT_FISICA', 'PUNT_RAZONA_CUANTITATIVO')
                df_1 =df_1.withColumnRenamed('PUNT_QUIMICA', 'PUNT_COMP_CIUDADANA')
            df_1 = cast_value(df_1,columns)
            df_1=df_1.select('COLE_NOMBRE_SEDE', expr(toSql(init_year,columns) + ' as Prom_Puntaje'))
            if(init_year==20191):
                print("RESULTADOS PRUEBAS " + str(int(init_year/10)))
                position_pro_calculation(df_1)
            init_year = init_year +1
    return 

In [7]:
#Execute the automation process
automation_saber_pro()

RESULTADOS PRUEBAS 2006
+-------+--------------------+-----------+------------------+
|Ranking|    cole_nombre_sede|Estudiantes| avg(Prom_Puntaje)|
+-------+--------------------+-----------+------------------+
|      1|INST ALBERTO MERA...|         16|         65.769375|
|      2|LIC NAVARRA      ...|         36|63.979691358024674|
|      3|COLEGIO MARYMOUNT...|         97|63.736769759450176|
|      4|LIC  DE CERVANTES...|         96|63.365648148148146|
|      5|COL LOS NOGALES  ...|         39|63.191196581196586|
|      6|COL  SAN BONIFACI...|         18|  62.8866049382716|
|      7|LIC CAMPO DAVID  ...|         19|62.862573099415194|
|      8|COL SAN CARLOS   ...|         94|62.718404255319165|
|      9|ASPAEN GIMNASIO L...|         20|62.715666666666664|
|     10|COLEGIO LEONARDO ...|          4| 62.53861111111111|
|     11|CENT. DE ENSEÑANZ...|         10|62.114333333333335|
|     12|COL SANTA FRANCIS...|         49| 62.09038548752835|
|     13|GIMN DE LOS CERRO...|         53| 61.

### Numeral C
Comparación del ranking obtenido en el numeral B con la clasificación por colegios.

Para comparar la clasificación de colegios con la de pruebas saber, se ordenarions los datos de los colegios de mayor a menor a traves de la variable INDICE_TOTAL proporcionada por el diccionario.
Este proceso se realizó a partir del año 2015.

In [8]:
# Configurations
ICFES_rdd=sc.textFile("./Colegios/SB11_CLASIFI_PLANTELES_20142/SB11-CLASIFI-PLANTELES-20142.txt")
header = ICFES_rdd.first()

#Base Methods
def replace(k,selector):
    word = k.split(selector)
    new_word = [k.split('"')[1] if '"' in k else k for k in word ]
    return new_word

def get_best_places(icfes_rdd_route, selector):
    ICFES_rdd=sc.textFile(icfes_rdd_route)
    values = ICFES_rdd.map(lambda k: replace(k,selector))
    log_df=values.toDF(header.split('"|"'))
    best = log_df.select("COLE_INST_NOMBRE",'INDICE_TOTAL', 'COLE_CATEGORIA"').sort(log_df["INDICE_TOTAL"].desc())
    best =best.collect()
    return best[1:10]

In [11]:
# Automation by years 2015-2019
def automation_best_places():
    init_year = 20151
    end_year=20182
    while (init_year<=end_year):
        if(init_year%2 == 0):
            try:
                file = f"./Colegios/SB11_CLASIFI_PLANTELES_{str(init_year)}/SB11-CLASIFI-PLANTELES-{str(init_year)}.txt"
                saber_pro=sc.textFile(file)
                header = saber_pro.first()
            except Exception as e:
                file = f"./Colegios/SB11_CLASIFI_PLANTELES_{str(init_year)}/SB11-CLASIFI-PLANTELES-{str(init_year)}.csv"
                saber_pro=sc.textFile(file)
                header = saber_pro.first()
            print("\n RESULTADOS PRUEBAS " + str(int(init_year/10)))
            selector = ',"|",' if (init_year == 20172) else "|"
            selector = "¬" if (init_year == 20182) else selector
            
            
            best_2 = get_best_places(file,selector)
            best = best_1 + best_2
            data = [[x.COLE_INST_NOMBRE, x.INDICE_TOTAL] for x in best]
            df = pd.DataFrame(data,columns=['Nombre Colegio','Puntaje'])
            print(df.sort_values(by=['Puntaje'], ascending=False))
            init_year = init_year + 9
        else:
            try:
                file = f"./Colegios/SB11_CLASIFI_PLANTELES_{str(init_year)}/SB11-CLASIFI-PLANTELES-{str(init_year)}.txt"
                saber_pro=sc.textFile(file)
                header = saber_pro.first()
            except Exception as e:
                file = f"./Colegios/SB11_CLASIFI_PLANTELES_{str(init_year)}/SB11-CLASIFI-PLANTELES-{str(init_year)}.csv"
                saber_pro=sc.textFile(file)
                header = saber_pro.first()
            selector = ',"|",' if (init_year == 20171) else "|"
            best_1 = get_best_places(file,selector)
            init_year = init_year + 1
    return


In [12]:
#Execute automation process
automation_best_places()


 RESULTADOS PRUEBAS 2015
                                       Nombre Colegio Puntaje
9         CAMBRIDGE SCHOOL (GIMN BILING PLAZA SESAMO)  0,9131
10                                INST ALBERTO MERANI  0,9129
11                                    LIC CAMPO DAVID  0,9127
0                                     COL LOS NOGALES   0,908
12                        COL NUEVO COLOMBO AMERICANO  0,9077
13                                        LIC NAVARRA   0,907
1                         COLEGIO BILINGUE DIANA OESE  0,9062
14                         GIMNASIO COLOMBO BRITANICO  0,9036
2                            COL LA QUINTA DEL PUENTE  0,9025
3                                        GIMN VERMONT  0,8993
15                                COL ANGLO AMERICANO  0,8993
4                         COL SAN JORGE DE INGLATERRA  0,8987
16  FUNDACION EDUCATIVA INSTITUTO EXPERIMENTAL JOS...  0,8978
17                                         COL REFOUS  0,8969
5                                      COL S

#### Análisis de Resultados
Al hacer un análisis de la información, se encentran leves diferencias en los puestos de los colegios a través de los años, vemos que en la mayoria de casos se muestran los mismos colegios con mayor puntaje.
Esta diferencia se da ya que el ICFES tiene en cuenta otras variables que en este caso no se pueden considerar; una de ellas es que por cada materia evaluada solo se toma el 70% de los mejores resultados obtenidos, adicionalmete, solo se toman los estudiantes que llevan mínimo 3 años estudiando en ese colegio y tambien se descartan los colegios que tengan menos de 9 estudiantes y menos del 80% de los estudiantes matriculados.
Estos factores desvían un poco el resultado obtenido con respecto al contrastado.

### Numeral D
Este numeral se encuentra cubierto en el numeral B.